In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Subset
from torchvision import transforms
from torchvision.datasets import MNIST, CIFAR10

from utils import training, evaluation
from models import bp_model, cnn_model

# 全局配置
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(DEVICE)
CRITERION = nn.CrossEntropyLoss()

# 数据集加载与预处理
def load_datasets():
    transform = transforms.Compose([transforms.ToTensor()])
    train_ds_map = {}
    test_ds_map  = {}
    train_ds_map['mnist'], test_ds_map['mnist']   = MNIST('./data', True, download=True,  transform=transform), MNIST('./data', False, download=True, transform=transform)
    train_ds_map['cifar10'], test_ds_map['cifar10'] = CIFAR10('./data', True, download=True, transform=transform), CIFAR10('./data', False, download=True, transform=transform)
    return train_ds_map, test_ds_map

train_ds_map, test_ds_map = load_datasets()
input_shape_map = { ds: train_ds_map[ds][0][0].shape for ds in train_ds_map }

# 统一测试集 DataLoader
def get_test_loader(dataset, batch_size=64):
    return DataLoader(test_ds_map[dataset], batch_size=batch_size, shuffle=False)

test_loader_map = { ds: get_test_loader(ds) for ds in test_ds_map }

# 交叉验证划分
def get_folds(dataset, n_splits=5, random_state=42):
    from sklearn.model_selection import KFold
    indices = np.arange(len(train_ds_map[dataset]))
    kf = KFold(n_splits=n_splits, shuffle=True, random_state=random_state)
    return list(kf.split(indices))

# 运行单次实验
def run_experiment(dataset, model_type, epochs=10, batch_size=32, **model_kwargs):
    test_accs = []
    folds = get_folds(dataset)
    for tr_idx, va_idx in folds:
        tr_loader = DataLoader(Subset(train_ds_map[dataset], tr_idx), batch_size=batch_size, shuffle=True)
        va_loader = DataLoader(Subset(train_ds_map[dataset], va_idx), batch_size=batch_size, shuffle=False)
        # 模型构建
        model_fn = bp_model.create_bp_model if model_type=='bp' else cnn_model.create_cnn_model
        model = model_fn(input_shape_map[dataset], num_classes=10).to(DEVICE)
        optimizer = optim.Adam(model.parameters(), lr=model_kwargs.get('lr', 1e-3))
        # 训练
        training.train_model(model, tr_loader, va_loader, epochs, DEVICE, CRITERION, optimizer)
        # 测试
        _, acc = evaluation.evaluate_model(model, test_loader_map[dataset], DEVICE, CRITERION)
        test_accs.append(acc)
    return np.array(test_accs)

models  = ['bp', 'cnn']
datasets = ['mnist', 'cifar10']
baseline = {}




cuda
Files already downloaded and verified
Files already downloaded and verified
